# 🎉 La Gran Fiesta de los Joins en Polars 🎉

## Efecto post-campaña o "conversion lag"
### Escenario 1:

- Una campaña se ejecuta hasta el 31 de enero
- Un usuario hace clic en un anuncio el 31 de enero a las 23:59
- El usuario completa la compra el 1 de febrero a las 00:01
- **No hay forma de atribuir la venta a la campaña porque click y venta no son del mismo día**

### Escenario 2:
- Tenemos una campaña de facebook que ha tenido ventas hasta el día de ayer y se apaga.
- El día de hoy alguien que guardo el enlace con la UTM de esa campaña de facebook realiza una compra.
- **La campaña ya está "apagada" cuando ocurre la conversión y no aparece en facebook para hoy, pero si en google analytics.**


### Por qué es un problema en Google Sheets:


En Sheets, típicamente tenemos los datos separados:

- Una tabla con datos de costos/clics por día
- Otra con datos de las ventas por día


Al usar VLOOKUP o BUSCARV para unir estos datos por fecha y nombre de campaña las conversiones que ocurren en días posteriores no se asocian con sus clics originales.

**Esto lleva a subestimar el ROAS de las campañas**

In [ ]:
import polars as pl
import numpy as np

## 🎭 Introducción: La Fiesta del Siglo
Imagina que estás organizando LA FIESTA DEL SIGLO y tienes dos listas:
- Una lista de invitados con sus preferencias de comida 🍽️
- Una lista de confirmaciones con el regalo que traerán 🎁

¡Necesitas juntar esta información de manera eficiente!
Aquí es donde entran nuestros amigos los JOINS.

In [ ]:
# Creamos nuestras listas de ejemplo
invitados = pl.DataFrame({
    'nombre': ['Ana Laura', 'Carlos', 'Mario', 'Elizabeth', 'Michelle'],
    'comida': ['Vegetariano', 'Todo', 'Vegano', 'Todo', 'Vegetariano'],
    'edad': [27, 34, 32, 30, 24]
})

confirmaciones = pl.DataFrame({
    'nombre': ['Ana Laura', 'Carlos', 'Andrés', 'Elizabeth', 'Josseline'],
    'regalo': ['Vino', 'Postre', 'Snacks', 'Música', 'Bebidas'],
    'invitados_extra': [0, 2, 1, 0, 1]
})

print("Lista de Invitados Original:")
print(invitados)
print("\nLista de Confirmaciones:")
print(confirmaciones)

Lista de Invitados Original:
shape: (5, 3)
┌───────────┬─────────────┬──────┐
│ nombre    ┆ comida      ┆ edad │
│ ---       ┆ ---         ┆ ---  │
│ str       ┆ str         ┆ i64  │
╞═══════════╪═════════════╪══════╡
│ Ana Laura ┆ Vegetariano ┆ 27   │
│ Carlos    ┆ Todo        ┆ 34   │
│ Mario     ┆ Vegano      ┆ 32   │
│ Elizabeth ┆ Todo        ┆ 30   │
│ Michelle  ┆ Vegetariano ┆ 24   │
└───────────┴─────────────┴──────┘

Lista de Confirmaciones:
shape: (5, 3)
┌───────────┬─────────┬─────────────────┐
│ nombre    ┆ regalo  ┆ invitados_extra │
│ ---       ┆ ---     ┆ ---             │
│ str       ┆ str     ┆ i64             │
╞═══════════╪═════════╪═════════════════╡
│ Ana Laura ┆ Vino    ┆ 0               │
│ Carlos    ┆ Postre  ┆ 2               │
│ Andrés    ┆ Snacks  ┆ 1               │
│ Elizabeth ┆ Música  ┆ 0               │
│ Josseline ┆ Bebidas ┆ 1               │
└───────────┴─────────┴─────────────────┘


## 🤝 1. INNER JOIN: Los que están en ambas listas
El INNER JOIN es como los invitados VIP: solo entran si están en AMBAS listas.
Es el más exclusivo de los joins, ¡no acepta nadie que no haya sido invitado o que no haya confirmado!

Ejemplo:
- Quieres saber qué regalo traerá cada invitado, pero solo de los que
  CONFIRMARON su asistencia.

In [ ]:
inner_party = invitados.join(
    confirmaciones,
    on='nombre',
    how='inner'
)

print("\n🌟 VIP (Inner Join):")
print(inner_party)


🌟 VIP (Inner Join):
shape: (3, 5)
┌───────────┬─────────────┬──────┬────────┬─────────────────┐
│ nombre    ┆ comida      ┆ edad ┆ regalo ┆ invitados_extra │
│ ---       ┆ ---         ┆ ---  ┆ ---    ┆ ---             │
│ str       ┆ str         ┆ i64  ┆ str    ┆ i64             │
╞═══════════╪═════════════╪══════╪════════╪═════════════════╡
│ Ana Laura ┆ Vegetariano ┆ 27   ┆ Vino   ┆ 0               │
│ Carlos    ┆ Todo        ┆ 34   ┆ Postre ┆ 2               │
│ Elizabeth ┆ Todo        ┆ 30   ┆ Música ┆ 0               │
└───────────┴─────────────┴──────┴────────┴─────────────────┘


## 👈 2. LEFT JOIN: Todos mis invitados (y sus regalos si confirmaron)

El LEFT JOIN es como decir: "Quiero saber de TODOS mis invitados originales,
y si confirmaron, también quiero saber qué regalo traerán"

Es como la tía que dice: "¿Y Fulanito? ¿No viene?" aunque no haya confirmado.

In [ ]:
left_party = invitados.join(
    confirmaciones,
    on='nombre',
    how='left'
)

print("\n👥 Todos los Invitados (Left Join):")
print(left_party)


👥 Todos los Invitados (Left Join):
shape: (5, 5)
┌───────────┬─────────────┬──────┬────────┬─────────────────┐
│ nombre    ┆ comida      ┆ edad ┆ regalo ┆ invitados_extra │
│ ---       ┆ ---         ┆ ---  ┆ ---    ┆ ---             │
│ str       ┆ str         ┆ i64  ┆ str    ┆ i64             │
╞═══════════╪═════════════╪══════╪════════╪═════════════════╡
│ Ana Laura ┆ Vegetariano ┆ 27   ┆ Vino   ┆ 0               │
│ Carlos    ┆ Todo        ┆ 34   ┆ Postre ┆ 2               │
│ Mario     ┆ Vegano      ┆ 32   ┆ null   ┆ null            │
│ Elizabeth ┆ Todo        ┆ 30   ┆ Música ┆ 0               │
│ Michelle  ┆ Vegetariano ┆ 24   ┆ null   ┆ null            │
└───────────┴─────────────┴──────┴────────┴─────────────────┘


## 👉 3. RIGHT JOIN: Todos los que confirmaron (estén o no invitados)

El RIGHT JOIN es como cuando alguien confirma que viene...
¡pero no recuerdas haberlo invitado! 😅

Muestra todos los que confirmaron asistencia, aunque no estén en la lista original.

In [ ]:
right_party = invitados.join(
    confirmaciones,
    on='nombre',
    how='right'
)

print("\n🎁 Todos los que Confirmaron (Right Join):")
print(right_party)


🎁 Todos los que Confirmaron (Right Join):
shape: (5, 5)
┌─────────────┬──────┬───────────┬─────────┬─────────────────┐
│ comida      ┆ edad ┆ nombre    ┆ regalo  ┆ invitados_extra │
│ ---         ┆ ---  ┆ ---       ┆ ---     ┆ ---             │
│ str         ┆ i64  ┆ str       ┆ str     ┆ i64             │
╞═════════════╪══════╪═══════════╪═════════╪═════════════════╡
│ Vegetariano ┆ 27   ┆ Ana Laura ┆ Vino    ┆ 0               │
│ Todo        ┆ 34   ┆ Carlos    ┆ Postre  ┆ 2               │
│ null        ┆ null ┆ Andrés    ┆ Snacks  ┆ 1               │
│ Todo        ┆ 30   ┆ Elizabeth ┆ Música  ┆ 0               │
│ null        ┆ null ┆ Josseline ┆ Bebidas ┆ 1               │
└─────────────┴──────┴───────────┴─────────┴─────────────────┘


## 🤗 4. OUTER JOIN: ¡Que no falte nadie!
El OUTER JOIN es como decir: "¡Mientras más seamos, mejor!"
Muestra TODOS, hayan o no confirmado, estén o no invitados originalmente.

Es la versión más inclusiva de los joins, ¡nadie se queda fuera!


In [ ]:
outer_party = invitados.join(
    confirmaciones,
    on='nombre',
    how='full' # también se conoce como full join
)

print("\n🌈 ¡Todos Incluidos! (Outer Join):")
print(outer_party)


🌈 ¡Todos Incluidos! (Outer Join):
shape: (7, 6)
┌───────────┬─────────────┬──────┬──────────────┬─────────┬─────────────────┐
│ nombre    ┆ comida      ┆ edad ┆ nombre_right ┆ regalo  ┆ invitados_extra │
│ ---       ┆ ---         ┆ ---  ┆ ---          ┆ ---     ┆ ---             │
│ str       ┆ str         ┆ i64  ┆ str          ┆ str     ┆ i64             │
╞═══════════╪═════════════╪══════╪══════════════╪═════════╪═════════════════╡
│ Ana Laura ┆ Vegetariano ┆ 27   ┆ Ana Laura    ┆ Vino    ┆ 0               │
│ Carlos    ┆ Todo        ┆ 34   ┆ Carlos       ┆ Postre  ┆ 2               │
│ null      ┆ null        ┆ null ┆ Andrés       ┆ Snacks  ┆ 1               │
│ Elizabeth ┆ Todo        ┆ 30   ┆ Elizabeth    ┆ Música  ┆ 0               │
│ null      ┆ null        ┆ null ┆ Josseline    ┆ Bebidas ┆ 1               │
│ Mario     ┆ Vegano      ┆ 32   ┆ null         ┆ null    ┆ null            │
│ Michelle  ┆ Vegetariano ┆ 24   ┆ null         ┆ null    ┆ null            │
└───────────┴──

### 🎯 Ejercicio Práctico: La Fiesta de Marketing

Ahora vamos a aplicar esto a un escenario de marketing digital.
Tenemos dos DataFrames:

1. campañas_activas:
   - campaña_id
   - nombre_campaña
   - presupuesto_diario
   - objetivo

2. rendimiento_campañas:
   - campaña_id
   - impresiones
   - clics
   - conversiones
   - costo

Crea estos DataFrames y practica cada tipo de join para:
1. Ver qué campañas activas tienen datos de rendimiento
2. Identificar campañas sin datos de rendimiento
3. Encontrar datos de rendimiento de campañas no activas
4. Obtener una vista completa de todas las campañas

In [ ]:
# Datos de ejemplo
campanas_activas = pl.DataFrame({
    'campaña_id': ['C001', 'C002', 'C003', 'C004', 'C005'],
    'nombre_campaña': ['Search_Brand', 'Display_Remark', 'Social_Awareness',
                      'Search_NonBrand', 'Display_Prosp'],
    'presupuesto_diario': [100, 150, 200, 300, 250],
    'objetivo': ['Conversiones', 'Awareness', 'Tráfico',
                'Conversiones', 'Awareness']
})

rendimiento_campanas = pl.DataFrame({
    'campaña_id': ['C001', 'C002', 'C006', 'C004', 'C007'],
    'impresiones': [10000, 25000, 15000, 30000, 20000],
    'clics': [500, 300, 400, 600, 350],
    'conversiones': [50, 20, 30, 45, 25],
    'costo': [95, 140, 180, 280, 200]
})

print(campanas_activas)
print(rendimiento_campanas)

shape: (5, 4)
┌────────────┬──────────────────┬────────────────────┬──────────────┐
│ campaña_id ┆ nombre_campaña   ┆ presupuesto_diario ┆ objetivo     │
│ ---        ┆ ---              ┆ ---                ┆ ---          │
│ str        ┆ str              ┆ i64                ┆ str          │
╞════════════╪══════════════════╪════════════════════╪══════════════╡
│ C001       ┆ Search_Brand     ┆ 100                ┆ Conversiones │
│ C002       ┆ Display_Remark   ┆ 150                ┆ Awareness    │
│ C003       ┆ Social_Awareness ┆ 200                ┆ Tráfico      │
│ C004       ┆ Search_NonBrand  ┆ 300                ┆ Conversiones │
│ C005       ┆ Display_Prosp    ┆ 250                ┆ Awareness    │
└────────────┴──────────────────┴────────────────────┴──────────────┘
shape: (5, 5)
┌────────────┬─────────────┬───────┬──────────────┬───────┐
│ campaña_id ┆ impresiones ┆ clics ┆ conversiones ┆ costo │
│ ---        ┆ ---         ┆ ---   ┆ ---          ┆ ---   │
│ str        ┆ i64    

In [ ]:
# Tu código aquí

## 💡 Bonus: Casos Especiales

1. CROSS JOIN: La fiesta se descontrola
   - Combina cada fila con todas las filas del otro DataFrame
   - Útil para crear todas las combinaciones posibles
   - ¡Cuidado! Puede crear muchos datos

2. SEMI JOIN: Los tímidos
   - Solo muestra filas del primer DataFrame que tienen match
   - No duplica columnas como el INNER JOIN
   - Útil para filtrar sin aumentar el tamaño de los datos

3. ANTI JOIN: Los rebeldes
   - Muestra las filas que NO tienen match
   - Perfecto para encontrar anomalías o datos faltantes

## 🎓 Consejos Finales

1. Siempre verifica tus datos antes y después del join (puedes usar la propiedad .shape para ver la cantidad de filas y columnas)
2. Considera el orden de las tablas (left vs right)
3. Revisa los valores nulos en el resultado
4. Piensa en el rendimiento con grandes conjuntos de datos
5. Siempre que sea posible usa IDs en vez de strings para la columna de union
